In [1]:
import os
import pandas as pd
import numpy as np
import rasterio

In [2]:
table = pd.read_csv(os.path.dirname(os.getcwd())+'\\OUTPUTS\\INTERMEDIATE\\classified_state_temp.csv')
state_name = table['State'].unique()[0]

C:\Users\harri\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (4,15,18,22,24) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
ds = rasterio.open(os.path.dirname(os.getcwd())+'\\OUTPUTS\\INTERMEDIATE\\'+state_name+'\\ID_NLCD_temp.tif')

In [4]:
ar =ds.read(1)

In [5]:
np.unique(ar)

array([100000000, 100000289, 100000863, ..., 900419473, 900419476,
       900419494])

In [6]:
ar = ar.astype(str)

In [7]:
f = lambda x: x[:2]
vfunc = np.vectorize(f)
lulc = vfunc(ar)

y = lambda x: x[-6:]
yfunc = np.vectorize(y)
parcels = yfunc(ar)

In [8]:
table['FFRC_LOCAL_ID'] = [(i+1) for i in list(table.index)]

In [9]:
cross = pd.read_csv(os.path.dirname(os.getcwd())+'\\OUTPUTS\\INTERMEDIATE\\'+state_name+'\\ID_Dictionary.csv')

In [15]:
len(cross)

420317

In [18]:
cross

,OID_,Value,Count,PRCLDMPID
0,0,1,42.0,100660192_161700952
1,1,2,19.0,100660192_161700951
2,2,3,30.0,100660192_161700950
3,3,4,37.0,100660192_161700949
4,4,5,32.0,100660192_161700948
...,...,...,...,...
420312,420312,423047,5.0,100660192_157280765
420313,420313,423048,25.0,100660192_157280766
420314,420314,423049,9.0,100660192_157280792
420315,420315,423050,9.0,100660192_157280767


In [27]:
df = cross.merge(table, on='PRCLDMPID', how = 'outer')

In [31]:
len(table['PRCLDMPID'].unique())

400624

In [ ]:
#JOIN_INDEX is ihe values from the cross table. Needs to be written into the main table

In [11]:
parcels = parcels.astype(float)

In [13]:
id_df = df[['Value', 'Own_Type']]
id_dict = {0:0}
for i in range(len(id_df['Value'])):
    id_dict[id_df['Value'][i]] = id_df['Own_Type'][i]

In [12]:
len(np.unique(parcels))

420318

In [14]:
z = lambda x: id_dict[x]
zfunc = np.vectorize(z)
ownerships = zfunc(parcels)

ValueError: cannot convert float NaN to integer

In [36]:
len(table['PRCLDMPID'].unique())

400624

In [37]:
path = r'D:\Documents\OwnershipMap\New_Script\Test_ENV\OUTPUTS\INTERMEDIATE'

In [39]:
temp = pd.read_csv(path+'\\temp.csv')

C:\Users\harri\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (2,13,16,20,22) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [40]:
state_temp = pd.read_csv(path+'\\state_temp.csv')

C:\Users\harri\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (3,14,17,21,23) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [41]:
fdt = pd.read_csv(path+'\\Full_Data_Table.csv')

C:\Users\harri\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (5,16,19,23,24,25,37) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [42]:
cst = pd.read_csv(path+'\\classified_state_temp.csv')

C:\Users\harri\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (4,15,18,22,24) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [46]:
len(fdt['PRCLDMPID'].unique())

398163